In [5]:
import sys
sys.path.append('/opt/anaconda3/lib/python3.12/site-packages')

from config import config
from snowflake_utils import SnowConnect
from datetime import date

today = date.today()
import logging as log

/opt/anaconda3/lib/python3.12/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: No module named 'snowflake.connector.nanoarrow_arrow_iterator'


In [28]:
import os
from cryptography.fernet import Fernet
from argon2 import PasswordHasher
from argon2.exceptions import VerifyMismatchError

# Initialize Argon2 password hasher
ph = PasswordHasher(
    time_cost=3,        # Number of iterations
    memory_cost=65536,  # Memory usage in KiB
    parallelism=4,      # Number of parallel threads
    hash_len=32,        # Length of the hash in bytes
    salt_len=16         # Length of the salt in bytes
)

# Initialize encryption key
def get_encryption_key():
    """Get or create encryption key"""
    key = os.getenv('ENCRYPTION_KEY')
    if not key:
        key = Fernet.generate_key()
        #print(key) #print key to save it as env variables
        log.warning("No encryption key found in environment. Generated new key. Please set ENCRYPTION_KEY environment variable.")
    return key
# Initialize Fernet cipher
cipher_suite = Fernet(get_encryption_key())

def encrypt_sensitive_data(data):
    """Encrypt sensitive data using Fernet"""
    if not data:
        return None
    return cipher_suite.encrypt(str(data).encode()).decode()

def decrypt_sensitive_data(encrypted_data):
    """Decrypt sensitive data using Fernet"""
    log.info(f'Encrypted data: {encrypted_data}')
    if not encrypted_data:
        return None
    try:
        return cipher_suite.decrypt(encrypted_data.encode()).decode()
    except Exception as e:
        log.error(f"Error decrypting data: {e}")
        return None

def encrypt_password(password):
    """Hash password using Argon2"""
    if not password:
        return None
    return ph.hash(str(password))

In [32]:
!pip install crewai

In [25]:
def encrypt_single_column(snow_conn, table, id_column, target_column):

    # Fetch ID and the column to encrypt
    snow_conn.cursor.execute(f"SELECT {id_column}, {target_column} FROM {table}")
    rows = snow_conn.cursor.fetchall()

    for row in rows:
        record_id = row[0]
        original_value = row[1]

        if original_value:
            encrypted_value = encrypt_sensitive_data(original_value)
            #encrypted_value = encrypt_password(original_value)
            update_query = f"""
                UPDATE {table}
                SET {target_column} = %s
                WHERE {id_column} = %s
            """
            snow_conn.cursor.execute(update_query, (encrypted_value, record_id))

    snow_conn.conn.commit()
    snow_conn.close()

def decrypt_single_column(snow_conn, table, id_column, target_column):

    # Fetch ID and the column to encrypt
    snow_conn.cursor.execute(f"SELECT {id_column}, {target_column} FROM {table}")
    rows = snow_conn.cursor.fetchall()

    for row in rows:
        record_id = row[0]
        original_value = row[1]

        if original_value:
            decrypted_value = decrypt_sensitive_data(original_value)
            #encrypted_value = encrypt_password(original_value)
            update_query = f"""
                UPDATE {table}
                SET {target_column} = %s
                WHERE {id_column} = %s
            """
            snow_conn.cursor.execute(update_query, (decrypted_value, record_id))

    snow_conn.conn.commit()
    snow_conn.close()

In [27]:
profile_connect = SnowConnect(config.PROFILE_TABLE_WAREHOUSE, config.PROFILE_TABLE_DATABASE, config.PROFILE_TABLE_SCHEMA)
encrypt_single_column(profile_connect, config.PROFILE_TABLE, 'UID', 'EMAIL')

In [ ]:
# ALTER TABLE your_table
# ALTER COLUMN PASSWORD VARCHAR(100),
# ALTER COLUMN EMAIL VARCHAR(100),
# ALTER COLUMN PHONE VARCHAR(100);

In [7]:
profile_connect = SnowConnect(config.PROFILE_TABLE_WAREHOUSE, config.PROFILE_TABLE_DATABASE, config.PROFILE_TABLE_SCHEMA)
add_columns_sql = f"""
ALTER TABLE {config.PROFILE_TABLE}
ALTER COLUMN EMAIL VARCHAR(100)
"""

profile_connect.cursor.execute(add_columns_sql)
profile_connect.conn.commit()

ProgrammingError: 040050 (22000): SQL compilation error: cannot change column EMAIL from type VARCHAR(16777216) to VARCHAR(100) because reducing the byte-length of a varchar is not supported.


In [4]:
profile_connect.close()

In [3]:

matching_connect = SnowConnect(config.MATCHING_TABLE_WAREHOUSE, config.MATCHING_TABLE_DATABASE, config.MATCHING_TABLE_SCHEMA)
add_columns_sql = f"""
ALTER TABLE {config.MATCHING_TABLE}
ADD COLUMN CONVERSATION_SID STRING
"""

matching_connect.cursor.execute(add_columns_sql)
matching_connect.conn.commit()
matching_connect.close()


In [4]:
# # Step 1: Connect to Snowflake for matching and profile tables
# matching_connect = SnowConnect(config.MATCHING_TABLE_WAREHOUSE, config.MATCHING_TABLE_DATABASE, config.MATCHING_TABLE_SCHEMA)
# profile_connect = SnowConnect(config.PROFILE_TABLE_WAREHOUSE, config.PROFILE_TABLE_DATABASE, config.PROFILE_TABLE_SCHEMA)

# # Step 2: Fetch all rows from matching table
# sql = f"SELECT UID1, UID2 FROM {config.MATCHING_TABLE}"
# matching_connect.cursor.execute(sql)
# rows = matching_connect.cursor.fetchall()

# # Step 3: For each row, fetch NAME1 and NAME2 from profile table and update
# for row in rows:
#     uid1, uid2 = row

#     # Fetch NAME for UID1
#     profile_connect.cursor.execute(f"SELECT NAME FROM {config.PROFILE_TABLE} WHERE UID = %s", (uid1,))
#     name1_row = profile_connect.cursor.fetchone()
#     name1 = name1_row[0] if name1_row else None

#     # Fetch NAME for UID2
#     profile_connect.cursor.execute(f"SELECT NAME FROM {config.PROFILE_TABLE} WHERE UID = %s", (uid2,))
#     name2_row = profile_connect.cursor.fetchone()
#     name2 = name2_row[0] if name2_row else None

#     # Update NAME1 and NAME2 in matching table
#     update_sql = f"""
#         UPDATE {config.MATCHING_TABLE}
#         SET NAME1 = %s, NAME2 = %s
#         WHERE UID1 = %s AND UID2 = %s
#     """
#     matching_connect.cursor.execute(update_sql, (name1, name2, uid1, uid2))

# # Step 4: Commit and close connections
# matching_connect.conn.commit()
# matching_connect.close()
# profile_connect.close()


In [5]:
# profile_connect = SnowConnect(config.PROFILE_TABLE_WAREHOUSE, config.PROFILE_TABLE_DATABASE, config.PROFILE_TABLE_SCHEMA)
# truncate = f'TRUNCATE TABLE {config.PROFILE_TABLE}'
# profile_connect.cursor.execute(truncate)
# profile_connect.conn.commit()
# profile_connect.close()

In [6]:
# matching_connect = SnowConnect(config.MATCHING_TABLE_WAREHOUSE, config.MATCHING_TABLE_DATABASE, config.MATCHING_TABLE_SCHEMA)
# truncate = f'TRUNCATE TABLE {config.PROFILE_TABLE}'
# matching_connect.cursor.execute(truncate)
# matching_connect.conn.commit()
# matching_connect.close()

In [9]:
profile_connect = SnowConnect(config.PROFILE_TABLE_WAREHOUSE, config.PROFILE_TABLE_DATABASE, config.PROFILE_TABLE_SCHEMA)
sql = f"SELECT * FROM {config.PROFILE_TABLE}"

# uid = 'c7acbe69-c22a-448b-b1f6-26fa0cd3d658'
# sql = f"DELETE FROM {config.PROFILE_TABLE} WHERE UID = '{uid}'"
profile_connect.cursor.execute(sql)

# Fetch all results
results = profile_connect.cursor.fetchall()
results
# profile_connect.conn.commit()
# profile_connect.close()

[('mumbai',
  'india',
  datetime.datetime(2025, 6, 12, 0, 52, 47),
  '1965-11-02',
  'srk.kingkhan@example.com',
  'male',
  'cricket,computer games,reading,collecting gadgets,traveling',
  'https://match-bucket-2025.s3.ohio-east-2.amazonaws.com/profile_pictures/d93d3907-422a-42c0-820a-819620df2107/image-0.png,https://match-bucket-2025.s3.ohio-east-2.amazonaws.com/profile_pictures/d93d3907-422a-42c0-820a-819620df2107/image-1.png',
  '28.6430858',
  datetime.datetime(2025, 6, 13, 4, 52, 24),
  '77.2192671',
  'shah rukh khan',
  '9000000001',
  datetime.time(14, 30),
  'd93d3907-422a-42c0-820a-819620df2107',
  'srkMagic123',
  'actor',
  'new delhi',
  'india',
  'https://match-bucket-2025.s3.ohio-east-2.amazonaws.com/notifications/d93d3907-422a-42c0-820a-819620df2107_notifications.yaml',
  None,
  None,
  None),
 ('mumbai',
  'india',
  datetime.datetime(2025, 6, 12, 0, 52, 50),
  '1988-05-01',
  'anushka.sharma@example.com',
  'female',
  'cooking,fitness,traveling,reading,photograph

In [12]:
len(results)

13

In [5]:
# uid1= 'cfed7071-dbe0-41e4-89b6-0cfae27b5163'
# uid2 = 'ac02ee31-0f65-48e8-9f94-22f574d85700'

matching_connect = SnowConnect(config.MATCHING_TABLE_WAREHOUSE, config.MATCHING_TABLE_DATABASE, config.MATCHING_TABLE_SCHEMA)
#sql = f"SELECT * FROM {config.MATCHING_TABLE} WHERE (UID1 = '{uid1}' AND UID2 ='{uid2}') OR (UID1 = '{uid1}' AND UID2= '{uid2}')"
sql = f"SELECT * FROM {config.MATCHING_TABLE}"
matching_connect.cursor.execute(sql)

# Fetch all results
results = matching_connect.cursor.fetchall()
results

[('67e78259-685f-42a8-98ad-e41bdb60a9f4',
  'aa08972c-4b6a-421d-b0d5-86a2638020c6',
  15.3,
  datetime.datetime(2025, 6, 8, 0, 59, 43),
  datetime.datetime(2025, 6, 10, 4, 29, 41),
  False,
  True,
  False,
  False,
  False,
  False,
  'alia bhatt',
  'shah rukh khan'),
 ('63047d6c-98a9-4690-a07f-5b2b5f360bbb',
  'aa08972c-4b6a-421d-b0d5-86a2638020c6',
  13.7,
  datetime.datetime(2025, 6, 8, 0, 59, 52),
  datetime.datetime(2025, 6, 11, 21, 31, 7),
  True,
  True,
  False,
  False,
  False,
  False,
  'priyanka chopra',
  'shah rukh khan'),
 ('f0dc5010-8812-46df-8299-465c1bd4fb6b',
  'aa08972c-4b6a-421d-b0d5-86a2638020c6',
  15.3,
  datetime.datetime(2025, 6, 8, 1, 0, 5),
  datetime.datetime(2025, 6, 11, 21, 40, 21),
  True,
  True,
  False,
  False,
  False,
  False,
  'sobhita dhulipala',
  'shah rukh khan'),
 ('c1c90e96-d17e-4bb7-8d92-c5f7d26afe4c',
  'aa08972c-4b6a-421d-b0d5-86a2638020c6',
  11.3,
  datetime.datetime(2025, 6, 8, 1, 0, 14),
  datetime.datetime(2025, 6, 10, 4, 10, 26)

In [ ]:
import subprocess


In [4]:
import pandas as pd

# Step 1: Create a connection to the profile table
profile_connect = SnowConnect(
    config.PROFILE_TABLE_WAREHOUSE,
    config.PROFILE_TABLE_DATABASE,
    config.PROFILE_TABLE_SCHEMA
)

# Step 2: Fetch all data from the profile table
sql = f"SELECT * FROM {config.PROFILE_TABLE}"
profile_connect.cursor.execute(sql)

# Step 3: Load result into a Pandas DataFrame
columns = [col[0] for col in profile_connect.cursor.description]  # get column names
rows = profile_connect.cursor.fetchall()  # get data

df_profile = pd.DataFrame(rows, columns=columns)

# Step 4: Optional – close connection
profile_connect.close()

today = date.today()
# Display or use the DataFrame
print(df_profile.head())

df_profile.to_csv(f'profile_{today}.csv', encoding='utf-8')


     CITY COUNTRY             CREATED         DOB  \
0  mumbai   india 2025-06-12 00:52:47  1965-11-02   
1  mumbai   india 2025-06-12 00:52:50  1988-05-01   
2  mumbai   india 2025-06-12 00:52:54  1993-03-15   
3  mumbai   india 2025-06-12 00:52:58  1982-07-18   
4  mumbai   india 2025-06-12 00:53:01  1992-05-31   

                         EMAIL  GENDER  \
0     srk.kingkhan@example.com    male   
1   anushka.sharma@example.com  female   
2       alia.bhatt@example.com  female   
3  priyanka.chopra@example.com  female   
4        sobhita.d@example.com  female   

                                             HOBBIES  \
0  cricket,computer games,reading,collecting gadg...   
1  cooking,fitness,traveling,reading,photography,...   
2   singing,acting,reading,drawing,traveling,hosting   
3  dancing,philanthropy,fitness,reading,traveling...   
4  dancing,reading,fashion,yoga,traveling,photogr...   

                                              IMAGES         LAT  \
0  https://match-bucket

In [1]:
import pandas as pd

# Step 1: Create a connection to the profile table
matching_connect = SnowConnect(
    config.MATCHING_TABLE_WAREHOUSE,
    config.MATCHING_TABLE_DATABASE,
    config.MATCHING_TABLE_SCHEMA
)

# Step 2: Fetch all data from the profile table
sql = f"SELECT * FROM {config.MATCHING_TABLE}"
matching_connect.cursor.execute(sql)

# Step 3: Load result into a Pandas DataFrame
columns = [col[0] for col in matching_connect.cursor.description]  # get column names
rows = matching_connect.cursor.fetchall()  # get data

df_matching = pd.DataFrame(rows, columns=columns)

# Step 4: Optional – close connection
matching_connect.close()

today = date.today()
# Display or use the DataFrame
print(df_matching.head())

df_matching.to_csv(f'matching_{today}.csv', encoding='utf-8')

NameError: name 'SnowConnect' is not defined

In [5]:
df_profile.to_csv('check_profile_table.csv', encoding='utf-8')

In [4]:
matching_connect = SnowConnect(config.MATCHING_TABLE_WAREHOUSE, config.MATCHING_TABLE_DATABASE, config.MATCHING_TABLE_SCHEMA)
#truncate = f'TRUNCATE TABLE {config.MATCHING_TABLE}'
update = f'UPDATE {config.MATCHING_TABLE} SET SCORE = ROUND(SCORE, 1)'
matching_connect.cursor.execute(update)
matching_connect.conn.commit()
matching_connect.close()

In [8]:
matching_connect = SnowConnect(config.MATCHING_TABLE_WAREHOUSE, config.MATCHING_TABLE_DATABASE, config.MATCHING_TABLE_SCHEMA)
sql = f"SELECT * FROM {config.MATCHING_TABLE}"
matching_connect.cursor.execute(sql)

# Fetch all results
results = matching_connect.cursor.fetchall()
results

[]

In [15]:
import os
import json
import subprocess
# Create accounts

with open('../docs/dummy_data_v2.json', 'r') as fp:
    data = json.load(fp)
len(data)

12

In [16]:
import os
import json
import subprocess
# Create accounts

with open('../docs/dummy_data_v2.json', 'r') as fp:
    data = json.load(fp)

for idx, i in enumerate(data[1:]):
    delimiter = f'" -F "'

    i = data[idx]
    image_commands=[]
    for pic in os.listdir(f'../docs/test-images/{idx}'):
        if pic == '.DS_Store':
            continue
        image_commands.append(f"images=@/Users/roshantiwari/src/KundaliMatch/docs/test-images/{idx}/{pic}")

    image_command_str = delimiter.join(image_commands)
    i = str(i)
    i = i.replace("'", '"')
    primary_command = f'''curl -X POST http://localhost:8080/account:create -F 'metadata={i}' -F "'''

    command = primary_command + image_command_str + '"'

    result = subprocess.run(command, shell = True, capture_output=True, text=True)
    print(f'{command}\n{result}')


curl -X POST http://localhost:8080/account:create -F 'metadata={"name": "Shah Rukh Khan", "phone": "9000000001", "password": "srkMagic123", "profession": "Actor", "birth_city": "New Delhi", "birth_country": "India", "email": "srk.kingkhan@example.com", "city": "Mumbai", "country": "India", "dob": "1965-11-02", "tob": "14:30", "gender": "male", "hobbies": ["cricket", "computer games", "reading", "collecting gadgets", "traveling"]}' -F "images=@/Users/roshantiwari/src/KundaliMatch/docs/test-images/0/2.png" -F "images=@/Users/roshantiwari/src/KundaliMatch/docs/test-images/0/1.png"
CompletedProcess(args='curl -X POST http://localhost:8080/account:create -F \'metadata={"name": "Shah Rukh Khan", "phone": "9000000001", "password": "srkMagic123", "profession": "Actor", "birth_city": "New Delhi", "birth_country": "India", "email": "srk.kingkhan@example.com", "city": "Mumbai", "country": "India", "dob": "1965-11-02", "tob": "14:30", "gender": "male", "hobbies": ["cricket", "computer games", "rea

In [ ]:
curl -X POST http://localhost:8080/account:update -F 'metadata={"uid": "9aae3cb1-f6dc-4201-b98d-eb25ca67e046", "tob": "12:40"}'

In [21]:
action = 'shJG'
action = action.lower()
action

'shjg'

In [ ]:
truncate = f'TRUNCATE TABLE {config.PROFILE_TABLE}'

In [3]:
import os
import boto3
# Configure AWS s3
s3 = boto3.client(
        's3',
        aws_access_key_id=os.getenv('S3_ACCESS_ID'),
        aws_secret_access_key=os.getenv('S3_ACCESS_KEY')
    )

BUCKET_NAME = os.getenv('BUCKET')
REGION = os.getenv("REGION")

In [4]:
from urllib.parse import urlparse
import base64
image_data_list = []
#path = 'https://match-bucket-2025.s3.ohio-east-2.amazonaws.com/profile_pictures/0f7314f7-f967-445a-95a7-1e1ded8dbaef/image-1.png'
path = 'https://match-bucket-2025.s3.ohio-east-2.amazonaws.com/notifications/f0dc5010-8812-46df-8299-465c1bd4fb6b_notifications.yaml'
parsed = urlparse(path)
key = parsed.path.lstrip('/')
s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=key)
yaml_file = s3_object['Body'].read()
# image_bytes = s3_object['Body'].read()
# encoded_image = base64.b64encode(image_bytes).decode('utf-8')
# image_data_list.append({
#     "filename": path,
#     "data": encoded_image
#     })


In [6]:
import yaml
notifications = yaml.safe_load(yaml_file)
notifications

[{'message': 'parshwa shah will not be recommended to you.',
  'updated': '2025-06-11 00:58:55'}]

In [5]:
key

'profile_pictures/0f7314f7-f967-445a-95a7-1e1ded8dbaef/image-1.png'

In [ ]:
# matchng_connect = SnowConnect(config.MATCHING_TABLE_WAREHOUSE, config.MATCHING_TABLE_DATABASE, config.MATCHING_TABLE_SCHEMA)
# rename_dict = {'WHATSAPP1' : 'ALIGN1','WHATSAPP2' : 'ALIGN2', 'INSTA1' : 'SKIP1', 'INSTA2' : 'SKIP2', 'SNAP1' : 'BLOCK1', 'SNAP2' : 'BLOCK2' }
# for old, new in rename_dict.items():

#     alter = f'ALTER TABLE {config.MATCHING_TABLE} RENAME COLUMN {old} TO {new}'
#     matchng_connect.cursor.execute(alter)
    
# matchng_connect.conn.commit()
# matchng_connect.close()


In [8]:
# matching_connect = SnowConnect(config.MATCHING_TABLE_WAREHOUSE, config.MATCHING_TABLE_DATABASE, config.MATCHING_TABLE_SCHEMA)
# SQL = f"UPDATE {config.MATCHING_TABLE} SET SCORE = ROUND(SCORE, 1)"
# matching_connect.cursor.execute(sql)
# matching_connect.conn.commit()
matching_connect.close()

In [4]:
matching_connect = SnowConnect(config.MATCHING_TABLE_WAREHOUSE, config.MATCHING_TABLE_DATABASE, config.MATCHING_TABLE_SCHEMA)
sql = f"SELECT column_name, data_type, is_nullable FROM information_schema.columns WHERE table_schema = '{config.MATCHING_TABLE_SCHEMA}' AND table_name = '{config.MATCHING_TABLE}'"
matching_connect.cursor.execute(sql)

# Fetch all results
results = matching_connect.cursor.fetchall()
matching_connect.close()
results

[('NAME2', 'TEXT', 'YES'),
 ('UID2', 'TEXT', 'YES'),
 ('BLOCK1', 'BOOLEAN', 'YES'),
 ('SCORE', 'FLOAT', 'YES'),
 ('UID1', 'TEXT', 'YES'),
 ('SKIP2', 'BOOLEAN', 'YES'),
 ('BLOCK2', 'BOOLEAN', 'YES'),
 ('ALIGN2', 'BOOLEAN', 'YES'),
 ('ALIGN1', 'BOOLEAN', 'YES'),
 ('UPDATED', 'TIMESTAMP_NTZ', 'YES'),
 ('SKIP1', 'BOOLEAN', 'YES'),
 ('CREATED', 'TIMESTAMP_NTZ', 'YES'),
 ('CONVERSATION_SID', 'TEXT', 'YES'),
 ('NAME1', 'TEXT', 'YES')]

In [5]:
profile_connect = SnowConnect(config.PROFILE_TABLE_WAREHOUSE, config.PROFILE_TABLE_DATABASE, config.PROFILE_TABLE_SCHEMA)
sql = f"SELECT column_name, data_type, is_nullable FROM information_schema.columns WHERE table_schema = '{config.PROFILE_TABLE_SCHEMA}' AND table_name = '{config.PROFILE_TABLE}'"
profile_connect.cursor.execute(sql)

# Fetch all results
results = profile_connect.cursor.fetchall()
results

[('IMAGES', 'TEXT', 'YES'),
 ('NOTIFICATIONS', 'TEXT', 'YES'),
 ('BIRTH_COUNTRY', 'TEXT', 'YES'),
 ('PASSWORD', 'TEXT', 'YES'),
 ('COUNTRY', 'TEXT', 'YES'),
 ('EMAIL', 'TEXT', 'YES'),
 ('DOB', 'TEXT', 'YES'),
 ('DESTINY_CHATS', 'TEXT', 'YES'),
 ('CITY', 'TEXT', 'YES'),
 ('TOB', 'TIME', 'YES'),
 ('PROFESSION', 'TEXT', 'YES'),
 ('LAT', 'TEXT', 'YES'),
 ('PHONE', 'TEXT', 'YES'),
 ('BIRTH_CITY', 'TEXT', 'YES'),
 ('HOBBIES', 'TEXT', 'YES'),
 ('LONG', 'TEXT', 'YES'),
 ('GENDER', 'TEXT', 'YES'),
 ('UID', 'TEXT', 'YES'),
 ('INITIATE_CHATS', 'TEXT', 'YES'),
 ('CREATED', 'TIMESTAMP_NTZ', 'YES'),
 ('LOGIN', 'TIMESTAMP_NTZ', 'YES'),
 ('NAME', 'TEXT', 'YES'),
 ('PREFERENCE_CHATS', 'TEXT', 'YES')]

In [8]:
sql = f"SELECT * FROM {config.PROFILE_TABLE}"
profile_connect.cursor.execute(sql)

# Fetch all results
results = profile_connect.cursor.fetchall()
results

[('mumbai',
  'india',
  datetime.datetime(2025, 6, 8, 0, 59, 17),
  '1965-11-02',
  'srk.kingkhan@example.com',
  'male',
  'cricket,computer games,reading,collecting gadgets,traveling',
  'https://match-bucket-2025.s3.ohio-east-2.amazonaws.com/profile_pictures/aa08972c-4b6a-421d-b0d5-86a2638020c6/image-0.png,https://match-bucket-2025.s3.ohio-east-2.amazonaws.com/profile_pictures/aa08972c-4b6a-421d-b0d5-86a2638020c6/image-1.png',
  '28.6430858',
  datetime.datetime(2025, 6, 10, 4, 34, 59),
  '77.2192671',
  'shah rukh khan',
  '9000000001',
  datetime.time(14, 30),
  'aa08972c-4b6a-421d-b0d5-86a2638020c6',
  'srkMagic123',
  'actor',
  'new delhi',
  'india',
  None,
  None),
 ('mumbai',
  'india',
  datetime.datetime(2025, 6, 8, 0, 59, 29),
  '1988-05-01',
  'anushka.sharma@example.com',
  'female',
  'cooking,fitness,traveling,reading,photography,yoga',
  'https://match-bucket-2025.s3.ohio-east-2.amazonaws.com/profile_pictures/ac02ee31-0f65-48e8-9f94-22f574d85700/image-0.png,https:/

In [10]:
result = results[-1]
obj = result[-1]
obj is None

True

In [3]:
uid = 'd72b7867-db49-45fb-9255-e3654d9b9c30'
matching_connect  = SnowConnect(config.MATCHING_TABLE_WAREHOUSE, config.MATCHING_TABLE_DATABASE, config.MATCHING_TABLE_SCHEMA)

sql_fetch = f"SELECT UID1, UID2, SCORE, UPDATED, ALIGN1, ALIGN2, SKIP1, SKIP2, BLOCK1, BLOCK2 FROM {config.MATCHING_TABLE} WHERE UID1 = '{uid}' OR UID2 = '{uid}'"
cursor_matching = matching_connect.conn.cursor()

cursor_matching.execute(sql_fetch)
results = cursor_matching.fetchall()
results[0]

('d72b7867-db49-45fb-9255-e3654d9b9c30',
 'user_007',
 18.545757603256128,
 datetime.datetime(2025, 6, 6, 23, 38, 12),
 False,
 False,
 False,
 False,
 False,
 False)

In [6]:
a = ['a', 'b', 'c', 'd']
a[1:3]

['b', 'c']

In [ ]:
uid = 'b'

recommended_id = result[0] if result[1] == uid else result[1]
recommended_id

'a'

In [ ]:
sql = F'ALTER TABLE {config.PROFILE_TABLE} ADD COLUMN PASSWORD TEXT'
profile_connect.cursor.execute(sql)


In [11]:
profile_connect.conn.commit()
profile_connect.close()

In [10]:
sql = F'ALTER TABLE {config.PROFILE_TABLE} ADD COLUMN BIRTH_COUNTRY TEXT'
profile_connect.cursor.execute(sql)

In [11]:
import time
t = current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [12]:
t2 = current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [16]:
type(t)

str

In [17]:
!pip install panchanga

ERROR: Could not find a version that satisfies the requirement panchanga (from versions: none)
ERROR: No matching distribution found for panchanga


In [9]:
select_sql = f"SELECT * FROM {config.PROFILE_TABLE} WHERE GENDER='male'"
profile_connect.cursor.execute(select_sql)

# Fetch all results
results = profile_connect.cursor.fetchall()

In [10]:
results[0]#[14]


('los angeles',
 'usa',
 datetime.datetime(2025, 5, 20, 9, 0),
 '{"day":3,"month":11,"year":1985}',
 'bob.smith@example.com',
 'male',
 '["gaming","cooking"]',
 '["bob1.png"]',
 '34.0522',
 datetime.datetime(2025, 6, 3, 18, 12),
 '-118.2437',
 'Bob Smith',
 '9876543210',
 datetime.time(8, 20),
 'user_002')

In [ ]:
import os
import boto3
from urllib.parse import urlparse

s3 = boto3.client('s3',
    aws_access_key_id=os.getenv('S3_ACCESS_ID'),
    aws_secret_access_key=os.getenv('S3_ACCESS_KEY'))

BUCKET_NAME = os.getenv('BUCKET')
REGION = os.getenv("REGION")  # e.g., 'us-east-1'

# s3.upload_file('./test-images/srk-1.png', BUCKET_NAME, 'srk-1.png')

# # Construct public URL
# url = f"https://{BUCKET_NAME}.s3.{REGION}.amazonaws.com/srk1.png"
chats_url = 'https://match-bucket-2025.s3.us-east-2.amazonaws.com/preference_chats/d93d3907-422a-42c0-820a-819620df2107_preference_chats.yaml'

chats_url_parsed = urlparse(chats_url)
chats_url = chats_url_parsed.path.lstrip('/')

try:
    # Download image from S3 as bytes
    s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=chats_url)
    yaml_content = s3_object['Body'].read()

    # Parse YAML
    chats = yaml.safe_load(yaml_content)
    if chats is None:
        chats = []

except Exception as e:
    log.warning(f'Unable to load chats url {chats_url}')
    chats = []

chats

[]

In [10]:
REGION

'us-east-2'

In [7]:
a = [1, 2]
a = str(a)

In [4]:
import time

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [6]:
type(current_time)

str

In [1]:
from geopy.geocoders import Nominatim

In [4]:
def get_lat_long(address):
    geolocator = Nominatim(user_agent="geo_locator")
    location = geolocator.geocode(address, timeout=10)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

In [5]:
lat, long = get_lat_long('new york, usa')

In [6]:
lat, long

(40.7127281, -74.0060152)

In [15]:
import yaml
def load_prompts(filepath: str) -> dict:
    with open(filepath, "r") as file:
        return yaml.safe_load(file)

In [17]:
prompts = load_prompts(config.PROMPTS_YAML)
name = 'q'
hobbies = 'sjcbjc'
system = prompts['system_prompt'].format(name ='a',hobbies =  'b')

In [18]:
system

'Start a conversation with the user a who enjoys hobbies: b.\nAsk friendly questions about their hobbies and follow up based on what they say.\nIf no history is provided then initiate a question based on the hobbies otherwise continue the existing conversation.\nTo initiate: If user has no hobbies then ask user something like "Hey a: I was wondering what do you do in your free time? This helps me find better matches for you"\nOr if the user has hobbies then ask for more details about their hobbies.\n\nKeep the conversation short that is dont ask more than 2 follow up questions.\nIf the user is interested in continuing the conversation, please do.\n\nEnd with "Thank you for chatting, this information helps us improve the recommendation algorithm to find better matches for you."\n\nPlease format the full conversation as a JSON object like this:\n      {\n          "messages": [\n              {"assistant": "Hi! You said you enjoy movies in your profile, Can you tell me aome of your favor